In [3]:
import pandas as pd

splits = {'train': 'synthetic_text_to_sql_train.snappy.parquet', 'test': 'synthetic_text_to_sql_test.snappy.parquet'}
df = pd.read_parquet("hf://datasets/gretelai/synthetic_text_to_sql/" + splits["train"])

In [4]:
df.head()

,id,domain,domain_description,sql_complexity,sql_complexity_description,sql_task_type,sql_task_type_description,sql_prompt,sql_context,sql,sql_explanation
0,5097,forestry,Comprehensive data on sustainable forest manag...,single join,"only one join (specify inner, outer, cross)",analytics and reporting,"generating reports, dashboards, and analytical...",What is the total volume of timber sold by eac...,"CREATE TABLE salesperson (salesperson_id INT, ...","SELECT salesperson_id, name, SUM(volume) as to...","Joins timber_sales and salesperson tables, gro..."
1,5098,defense industry,"Defense contract data, military equipment main...",aggregation,"aggregation functions (COUNT, SUM, AVG, MIN, M...",analytics and reporting,"generating reports, dashboards, and analytical...",List all the unique equipment types and their ...,CREATE TABLE equipment_maintenance (equipment_...,"SELECT equipment_type, SUM(maintenance_frequen...",This query groups the equipment_maintenance ta...
2,5099,marine biology,"Comprehensive data on marine species, oceanogr...",basic SQL,basic SQL with a simple select statement,analytics and reporting,"generating reports, dashboards, and analytical...",How many marine species are found in the South...,"CREATE TABLE marine_species (name VARCHAR(50),...",SELECT COUNT(*) FROM marine_species WHERE loca...,This query counts the number of marine species...
3,5100,financial services,Detailed financial data including investment s...,aggregation,"aggregation functions (COUNT, SUM, AVG, MIN, M...",analytics and reporting,"generating reports, dashboards, and analytical...",What is the total trade value and average pric...,"CREATE TABLE trade_history (id INT, trader_id ...","SELECT trader_id, stock, SUM(price * quantity)...",This query calculates the total trade value an...
4,5101,energy,Energy market data covering renewable energy s...,window functions,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...",analytics and reporting,"generating reports, dashboards, and analytical...",Find the energy efficiency upgrades with the h...,"CREATE TABLE upgrades (id INT, cost FLOAT, typ...","SELECT type, cost FROM (SELECT type, cost, ROW...",The SQL query uses the ROW_NUMBER function to ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   id                          100000 non-null  int32 
 1   domain                      100000 non-null  object
 2   domain_description          100000 non-null  object
 3   sql_complexity              100000 non-null  object
 4   sql_complexity_description  100000 non-null  object
 5   sql_task_type               100000 non-null  object
 6   sql_task_type_description   100000 non-null  object
 7   sql_prompt                  100000 non-null  object
 8   sql_context                 100000 non-null  object
 9   sql                         100000 non-null  object
 10  sql_explanation             100000 non-null  object
dtypes: int32(1), object(10)
memory usage: 8.0+ MB


In [7]:
!pip uninstall -y unsloth peft

!pip install unsloth trl peft accelerate bitsandbytes

Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 758.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [8]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [9]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Meta-Llama-3.1-8B"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [10]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [19]:
# # Alpaca prompt template
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# def format_to_alpaca(row):
#     """Convert dataset row to Alpaca format"""
#     instruction = "Generate a SQL query to answer the following question based on the provided database schema."

#     # Combine the question and schema as input
#     input_text = f"Question: {row['sql_prompt']}\n\nDatabase Schema:\n{row['sql_context']}"

#     # The SQL query is the response
#     response = row['sql']

#     # Format using Alpaca template
#     text = alpaca_prompt.format(instruction, input_text, response)
#     return text

# # Apply formatting to dataset
# print("Formatting dataset to Alpaca format...")
# df['text'] = df.apply(format_to_alpaca, axis=1)

# # Convert to Hugging Face Dataset format
# from datasets import Dataset

# dataset = Dataset.from_pandas(df[['text']])
# print(f"✓ Training samples: {len(dataset)}")
# print("\n" + "="*60)
# print("SAMPLE FORMATTED PROMPT:")
# print("="*60)
# print(dataset[0]['text'][:600] + "...\n")

Formatting dataset to Alpaca format...
✓ Training samples: 100000

SAMPLE FORMATTED PROMPT:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a SQL query to answer the following question based on the provided database schema.

### Input:
Question: What is the total volume of timber sold by each salesperson, sorted by salesperson?

Database Schema:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_...



In [20]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# print("Initializing trainer...")

# # Training arguments optimized for Unsloth
# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = False,  # Can make training 5x faster for short sequences
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         num_train_epochs = 1,  # Use this for full training (recommended)
#         # max_steps = 60,  # OR use this for quick testing
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 10,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         save_strategy = "epoch",
#         save_total_limit = 2,
#     ),
# )

# print("✓ Trainer initialized successfully!\n")

Initializing trainer...


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

✓ Trainer initialized successfully!



In [21]:
# print("="*60)
# print("STARTING TRAINING")
# print("="*60)
# print(f"Total samples: {len(dataset)}")
# print(f"Batch size: 2")
# print(f"Gradient accumulation: 4")
# print(f"Effective batch size: 8")
# print(f"Estimated time: 2-4 hours on T4 GPU")
# print("="*60 + "\n")

# trainer_stats = trainer.train()

# print("\n" + "="*60)
# print("✓✓✓ TRAINING COMPLETED! ✓✓✓")
# print("="*60)
# print(f"Training loss: {trainer_stats.training_loss:.4f}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


STARTING TRAINING
Total samples: 100000
Batch size: 2
Gradient accumulation: 4
Effective batch size: 8
Estimated time: 2-4 hours on T4 GPU



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 12,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: onenishitrathod (onenishitrathod-humber-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.149200
20,0.536200
30,0.461400
40,0.440000
50,0.435400
60,0.436700
70,0.431000


KeyboardInterrupt: 

In [22]:
print("Preparing small training dataset for quick demo...")

# Use only 500 samples instead of 100,000!
df_small = df.head(500)

# Apply Alpaca formatting
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def format_to_alpaca(row):
    instruction = "Generate a SQL query to answer the following question based on the provided database schema."
    input_text = f"Question: {row['sql_prompt']}\n\nDatabase Schema:\n{row['sql_context']}"
    response = row['sql']
    return alpaca_prompt.format(instruction, input_text, response)

df_small['text'] = df_small.apply(format_to_alpaca, axis=1)

from datasets import Dataset
dataset = Dataset.from_pandas(df_small[['text']])

print(f"✓ Training on {len(dataset)} samples (reduced from 100,000)")
print("✓ Estimated training time: 15-20 minutes\n")

Preparing small training dataset for quick demo...
✓ Training on 500 samples (reduced from 100,000)
✓ Estimated training time: 15-20 minutes



/tmp/ipython-input-2314547219.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['text'] = df_small.apply(format_to_alpaca, axis=1)


In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

print("Setting up FAST trainer...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,  # Changed to True - makes training 5x faster!
    args = TrainingArguments(
        per_device_train_batch_size = 4,  # Increased from 2
        gradient_accumulation_steps = 2,  # Decreased from 4
        warmup_steps = 10,
        num_train_epochs = 3,  # 3 epochs on small data = better results
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy = "no",  # Don't save checkpoints during training
    ),
)

print("✓ Fast trainer configured!\n")

Setting up FAST trainer...


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

✓ Fast trainer configured!



In [24]:
print("="*60)
print("STARTING FAST TRAINING FOR DEMO")
print("="*60)
print(f"Samples: {len(dataset)}")
print(f"Epochs: 3")
print(f"Batch size: 4")
print(f"Packing: Enabled (5x faster)")
print(f"Time estimate: 15-20 minutes")
print("="*60 + "\n")

import time
start_time = time.time()

trainer_stats = trainer.train()

end_time = time.time()
training_time = (end_time - start_time) / 60

print("\n" + "="*60)
print("✓✓✓ TRAINING COMPLETED! ✓✓✓")
print("="*60)
print(f"Training loss: {trainer_stats.training_loss:.4f}")
print(f"Time taken: {training_time:.1f} minutes")

The model is already on multiple devices. Skipping the move to device specified in `args`.


STARTING FAST TRAINING FOR DEMO
Samples: 500
Epochs: 3
Batch size: 4
Packing: Enabled (5x faster)
Time estimate: 15-20 minutes



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 189
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
5,0.428600
10,0.431000
15,0.447400
20,0.423500
25,0.421500
30,0.404300
35,0.425500
40,0.391000
45,0.437400
50,0.430700



✓✓✓ TRAINING COMPLETED! ✓✓✓
Training loss: 0.3311
Time taken: 18.5 minutes


In [25]:
print("\n" + "="*60)
print("TESTING MODEL")
print("="*60)

FastLanguageModel.for_inference(model)

def test_sql(question, schema):
    instruction = "Generate a SQL query to answer the following question based on the provided database schema."
    input_text = f"Question: {question}\n\nDatabase Schema:\n{schema}"
    prompt = alpaca_prompt.format(instruction, input_text, "")

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.3)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### Response:" in result:
        return result.split("### Response:")[-1].strip()
    return result

# Quick test
test_sample = df_small.iloc[0]
print(f"\nQuestion: {test_sample['sql_prompt'][:80]}...")
print(f"Schema: {test_sample['sql_context'][:100]}...")
print(f"\nExpected: {test_sample['sql'][:80]}...")
generated = test_sql(test_sample['sql_prompt'], test_sample['sql_context'])
print(f"Generated: {generated[:80]}...")


TESTING MODEL

Question: What is the total volume of timber sold by each salesperson, sorted by salespers...
Schema: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (sale...

Expected: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN ...
Generated: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN ...


In [ ]:
# print("\n" + "="*60)
# print("TESTING THE MODEL")
# print("="*60)

#  #Enable fast inference mode
# FastLanguageModel.for_inference(model)

# def test_sql_generation(question, schema):
#     """Test the fine-tuned model with Alpaca format"""
#     instruction = "Generate a SQL query to answer the following question based on the provided database schema."
#     input_text = f"Question: {question}\n\nDatabase Schema:\n{schema}"

#     # Format prompt (without response for inference)
#     prompt = alpaca_prompt.format(instruction, input_text, "")

#     inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=256,
#         temperature=0.3,
#         top_p=0.9,
#         do_sample=True,
#         pad_token_id=tokenizer.eos_token_id
#     )

#     result = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Extract only the response part after "### Response:"
#     if "### Response:" in result:
#         sql_query = result.split("### Response:")[-1].strip()
#     else:
#         sql_query = result

#     return sql_query

# # Test with samples from the dataset
# print("\nRunning 3 test queries...\n")

# for i in range(3):
#     test_sample = df.iloc[i]
#     print(f"\n{'='*60}")
#     print(f"TEST {i+1}")
#     print('='*60)
#     print(f"Question: {test_sample['sql_prompt']}")
#     print(f"\nSchema: {test_sample['sql_context'][:150]}...")
#     print(f"\n✓ Expected SQL:\n{test_sample['sql']}")

#     generated = test_sql_generation(test_sample['sql_prompt'], test_sample['sql_context'])
#     print(f"\n✓ Generated SQL:\n{generated}")

In [ ]:
print("\n" + "="*60)
print("SAVING MODEL FOR OLLAMA")
print("="*60)

# Save directly as GGUF - skip other formats to save time
model.save_pretrained_gguf("demo_model", tokenizer, quantization_method="q4_k_m")
print("✓ Model saved as GGUF (Q4_K_M - optimized for speed)")


SAVING MODEL FOR OLLAMA
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [11:04<00:00, 166.09s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [09:47<00:00, 146.93s/it]


Unsloth: Merge process complete. Saved to `/content/demo_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages


In [ ]:
# print("\n" + "="*60)
# print("SAVING MODEL")
# print("="*60)

# # Save LoRA adapters only (small size ~150MB)
# model.save_pretrained("text_to_sql_lora")
# tokenizer.save_pretrained("text_to_sql_lora")
# print("✓ LoRA adapters saved to: text_to_sql_lora/")

# # Save merged 16-bit model
# model.save_pretrained_merged("text_to_sql_merged", tokenizer, save_method="merged_16bit")
# print("✓ Merged 16-bit model saved to: text_to_sql_merged/")

In [ ]:
# print("\n" + "="*60)
# print("CONVERTING TO GGUF FORMAT FOR OLLAMA")
# print("="*60)

# # Save as GGUF with different quantization levels
# # q8_0 = high quality, larger size
# model.save_pretrained_gguf("text_to_sql_gguf", tokenizer, quantization_method="q8_0")
# print("✓ GGUF Q8_0 model saved to: text_to_sql_gguf/")

# # Optional: Also save q4_0 for smaller size
# model.save_pretrained_gguf("text_to_sql_gguf_q4", tokenizer, quantization_method="q4_k_m")
# print("✓ GGUF Q4_K_M model saved to: text_to_sql_gguf_q4/ (smaller, faster)")

In [ ]:
# print("\n" + "="*60)
# print("PREPARING FILES FOR DOWNLOAD")
# print("="*60)

# from google.colab import files
# import os

# # Create zip files
# print("\nCreating zip archives...")
# !zip -r text_to_sql_q8.zip text_to_sql_gguf/
# !zip -r text_to_sql_q4.zip text_to_sql_gguf_q4/
# !zip -r text_to_sql_lora.zip text_to_sql_lora/

# print("\n" + "="*60)
# print("DOWNLOAD OPTIONS:")
# print("="*60)
# print("1. text_to_sql_q8.zip - High quality (recommended)")
# print("2. text_to_sql_q4.zip - Smaller size, faster inference")
# print("3. text_to_sql_lora.zip - LoRA adapters only")
# print("="*60 + "\n")

# # Download the main model (Q8)
# print("Downloading high-quality Q8 model...")
# files.download('text_to_sql_q8.zip')

# print("\n✓✓✓ ALL STEPS COMPLETED SUCCESSFULLY! ✓✓✓")
# print("\nNext steps:")
# print("1. Extract the downloaded zip file")
# print("2. Follow the Ollama setup guide")
# print("3. Run your demo!")

In [1]:
print("\n" + "="*60)
print("PREPARING FOR DOWNLOAD")
print("="*60)

!zip -r demo_model.zip demo_model/

from google.colab import files
print("\nDownloading model...")
files.download('demo_model.zip')

print("\n" + "="*60)
print("✓✓✓ READY FOR DEMO! ✓✓✓")
print("="*60)
print(f"Total time: ~{training_time:.1f} minutes")
print("Model size: ~4-5 GB (Q4 quantization)")
print("\nNext: Extract zip and run Ollama setup")
print("="*60)


PREPARING FOR DOWNLOAD
  adding: demo_model/ (stored 0%)
  adding: demo_model/tokenizer_config.json (deflated 96%)
  adding: demo_model/model-00003-of-00004.safetensors (deflated 21%)
  adding: demo_model/.cache/ (stored 0%)
  adding: demo_model/.cache/huggingface/ (stored 0%)
  adding: demo_model/.cache/huggingface/.gitignore (stored 0%)
  adding: demo_model/.cache/huggingface/download/ (stored 0%)
  adding: demo_model/.cache/huggingface/download/model-00004-of-00004.safetensors.metadata (deflated 30%)
  adding: demo_model/.cache/huggingface/download/model-00002-of-00004.safetensors.lock (stored 0%)
  adding: demo_model/.cache/huggingface/download/model-00001-of-00004.safetensors.lock (stored 0%)
  adding: demo_model/.cache/huggingface/download/model-00004-of-00004.safetensors.lock (stored 0%)
  adding: demo_model/.cache/huggingface/download/model.safetensors.index.json.metadata (deflated 27%)
  adding: demo_model/.cache/huggingface/download/model-00003-of-00004.safetensors.metadata 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ READY FOR DEMO! ✓✓✓


NameError: name 'training_time' is not defined